# Data Driven Business project

-
-
- Rick van der Kleij
- Mathijs de Jong (V2B)

In [412]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from cleaning import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor, plot_tree


### Importeren dataset

In [413]:
# Load the data
df = load_data('sap_storing_data_hu_project.csv')

# Data understanding

In [ ]:
df.describe()

# Data cleaning

In [ ]:
# Define the columns to drop
columns_to_drop = ['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld',
                   'stm_scenario_mon', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd',
                   'stm_aanntpl_ddt', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt',
                   'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz',
                   'stm_veroorz_groep', 'stm_veroorz_code', 'stm_veroorz_tekst_kort', 'stm_effect', 'stm_afspr_aanvangddt',
                   'stm_mon_eind_ddt', 'stm_mon_vhdsincident', 'stm_dir_betrok_tr', 'stm_aangelegd_dd', 'stm_aangelegd_tijd',
                   'stm_mon_begindatum', 'stm_mon_begintijd', 'stm_progfh_gw_datum', 'stm_mon_eind_datum', 'stm_mon_eind_tijd',
                   'stm_controle_dd', 'stm_akkoord_mon_toewijz', 'stm_status_sapnaarmon', 'stm_fact_jn', 'stm_akkoord_melding_jn',
                   'stm_afsluit_ddt', 'stm_afsluit_dd', 'stm_afsluit_tijd', 'stm_rec_toegev_ddt', 'stm_hinderwaarde',
                   'stm_actie', 'stm_standplaats', 'stm_status_gebr', 'stm_wbi_nummer', 'stm_projnr', 'stm_historie_toelichting',
                   'stm_schade_verhaalb_jn', 'stm_schadenr', 'stm_schade_status_ga', 'stm_schade_statusdatum', 'stm_relatiervo_vorig',
                   'stm_relatiervo_volgend', 'stm_relatiervo', 'stm_afspr_func_hersteldd', 'stm_afspr_func_hersteltijd',
                   'stm_sorteerveld', 'stm_rapportage_maand', 'stm_rapportage_jaar', 'stm_x_bron_publ_dt', 'stm_x_bron_bestandsnaam',
                   'stm_x_bron_arch_dt', 'stm_x_actueel_ind', 'stm_x_run_id', 'stm_x_bk', 'stm_x_start_sessie_dt', 'stm_x_vervallen_ind']

# Drop the unnecessary columns
df = drop_columns(df, columns_to_drop)

# Clean the data (handle missing values, remove columns with excessive NaNs)
df, avg_list, mode_list = clean_data(df)

# Filter de data op rijen met een totale_functiehersteltijd van tussen de 5 mins en 8 uur.
df = filter_data(df)

# Save the cleaned data to a new CSV (optional)
save_data(df, 'final_db_cleaned.csv')

# Display results (optional)
print("Data cleaning complete.")
print("Columns filled with averages: ", avg_list)
print("Columns filled with mode: ", mode_list)
print("Cleaned dataframe shape: ", df.shape)


In [416]:
# Converteer prognose invoer kolommen naar bruikbare datetime objecten.
df['stm_progfh_in_invoer_dat'] = pd.to_datetime(df['stm_sap_meld_ddt'].dt.date)
df['stm_progfh_in_invoer_tijd'] = pd.to_timedelta(df['stm_progfh_in_invoer_tijd'])

# Maak een nieuwe kolom voor het tijdstip waarop de aannemer zijn prognose invult.
df['prognose_invoer_tijdstip'] = df['stm_progfh_in_invoer_dat'] + df['stm_progfh_in_invoer_tijd']

# Maak een nieuwe kolom aan voor de targetvariabele, de tijd tussen het invullen van de prognose door de aannemer en functieherstel.
df['target'] = df['totale_functiehersteltijd'] - (df['prognose_invoer_tijdstip'] - df['stm_sap_meld_ddt'])
# Zorg ervoor dat de targetvariabele altijd 0 dagen heeft (de berekeningen hierboven kunnen leiden tot een target value dat 1 dag te hoog is.)
df['target'] = df['target'].apply(lambda x: pd.Timedelta(hours=x.seconds // 3600, minutes=(x.seconds // 60) % 60, seconds=x.seconds % 60))

# Verwijder alle rijen met een negatieve target. In deze gevallen heeft de aannemer zijn prognose na het functieherstel ingevuld, en is de data niet te gebruiken om op te trainen.
df = df[df['target'] >= pd.Timedelta(0)]

In [ ]:
# Testen
df[['stm_sap_meld_ddt', 'stm_fh_ddt', 'prognose_invoer_tijdstip', 'totale_functiehersteltijd', 'target']].sample(10)

# Baseline model

In [418]:
# Updated function to handle both scalar and array return types
def mode_per_group(group):
    mode_result = stats.mode(group, keepdims=True)  # Ensure it returns in an array-like format
    return mode_result.mode[0]  # Safely access the mode value

# Calculate baseline (mode) for each oorzaak code
baseline_modes = df.groupby('stm_oorz_code')['stm_fh_duur'].apply(mode_per_group).reset_index()
baseline_modes.columns = ['stm_oorz_code', 'baseline_fh_duur']

# Merge baseline values with the original DataFrame
df = df.merge(baseline_modes, on='stm_oorz_code')

# Calculate the difference between actual time and baseline (mode)
df['difference'] = df['stm_fh_duur'] - df['baseline_fh_duur']

In [ ]:
# Mean Absolute Error (MAE)
mae = df['difference'].abs().mean()
print(f'Mean Absolute Error (MAE): {mae}')


In [ ]:
# Check if the baseline prediction matches the actual time
df['correct_prediction'] = df['stm_fh_duur'] == df['baseline_fh_duur']

# Calculate the accuracy in percentage
accuracy = df['correct_prediction'].mean() * 100

# Print the accuracy
print(f'Accuracy of the baseline model: {accuracy:.2f}%')


In [421]:
# Verwijder spaties aan het begin en einde van de waarden
df['stm_progfh_in_duur_clean'] = df['stm_progfh_in_duur'].str.strip()

# Vervang ongeldige waarden door NaN en converteer naar numeriek
df['stm_progfh_in_duur_clean'] = pd.to_numeric(df['stm_progfh_in_duur_clean'], errors='coerce')

# Vul NaN-waarden in met de gemiddelde waarde (zonder inplace=True)
df['stm_progfh_in_duur_clean'] = df['stm_progfh_in_duur_clean'].fillna(df['stm_progfh_in_duur_clean'].mean())

In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code']]
y = df['stm_fh_duur']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineair regressiemodel aanmaken
model = LinearRegression()

# Train het model
model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred = model.predict(X_test)

# Bereken de evaluatiestatistieken
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}') 
print(f'R² waarde: {r2_dt}') 

# Plot de voorspellingen
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, color='blue', label='Voorspellingen')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', linestyle='--', label='Ideale voorspelling')
plt.title('Voorspellingen vs. Werkelijke Waarden')
plt.xlabel('Werkelijke Waarden')
plt.ylabel('Voorspellingen')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code']]
y = df['stm_fh_duur']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Regressor aanmaken
dt_model = DecisionTreeRegressor(random_state=42)

# Train het model
dt_model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred_dt = dt_model.predict(X_test)

# Bereken de evaluatiestatistieken
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print(f'Mean Squared Error van Decision Tree: {mse_dt}') 
print(f'R² waarde van Decision Tree: {r2_dt}') 

# Visualiseer de Decision Tree
# plt.figure(figsize=(12, 8))
# plot_tree(dt_model, feature_names=X.columns, filled=True)
# plt.title('Decision Tree voor stm_fh_duur')
# plt.show()


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code']]
y = df['stm_fh_duur']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Regressor aanmaken met beperkte diepte
dt_model = DecisionTreeRegressor(max_depth=3, random_state=42)

# Train het model
dt_model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred_dt = dt_model.predict(X_test)

# Bereken de evaluatiestatistieken
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print(f'Mean Squared Error van Decision Tree: {mse_dt}') 
print(f'R² waarde van Decision Tree: {r2_dt}') 

# Visualiseer de Decision Tree
plt.figure(figsize=(12, 8))
plot_tree(dt_model, feature_names=X.columns, filled=True)
plt.title('Decision Tree voor stm_fh_duur (max_depth=3)')
plt.show()
